In [1]:
# Importing the warnings module to manage runtime warnings and  Suppressing all warnings during runtime
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing necessary libraries
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

# Initializing variables for storing words, classes, and documents
words=[]
classes = []
documents = []

ignore_words = ['?', '!']
# Reading the JSON file containing intents data
data_file = open('latestintent.json', encoding="utf8").read()
intents = json.loads(data_file)

2024-04-19 22:16:51.890036: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
#display intents
intents

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, thanks for asking',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'tag': 'options',
   'patterns': ['How you could help me?',
    'What you can do

In [4]:
nltk.download('punkt') # Download the required NLTK data

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/adithyasanthilal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# Loop through each intent and its patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word in the pattern
        w = nltk.word_tokenize(pattern)
        words.extend(w)  # Extend the words list with tokenized words
       # Append the tokenized pattern and its intent tag to documents
        documents.append((w, intent['tag']))
     # Add the intent tag to the classes list if it's not already there
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [6]:
# Download the WordNet corpus for lemmatization
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/adithyasanthilal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
# Lemmatize words, convert to lowercase, and remove words in ignore_words list
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
# Remove duplicates, sort, and convert to a sorted list of unique words (vocabulary)
words = sorted(list(set(words)))
# Sort the classes (intents)
classes = sorted(list(set(classes)))
# Print statistics about the documents, classes, and words
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)
# Save the words and classes to pickle files for later use
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [8]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Assume you have defined 'words', 'classes', and 'documents' appropriately

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # Create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# Shuffle training data
random.shuffle(training)

# Separate input (X) and output (Y)
train_x = np.array([t[0] for t in training])  # Extract bag of words as input
train_y = np.array([t[1] for t in training])  # Extract output row as target

# Pad sequences to ensure uniform length
train_x = pad_sequences(train_x, padding='post', maxlen=88)

# Print shape of training data
print("Training data shape:", train_x.shape)

# Define model architecture
model = Sequential()
model.add(Dense(128, input_shape=(train_x.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save model
model.save('chatbot_model.h5')
print("Model created and saved.")


Training data shape: (47, 88)
Epoch 1/200
10/10 [==============================] - 1s 9ms/step - loss: 2.2319 - accuracy: 0.1064
Epoch 2/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1806 - accuracy: 0.1702
Epoch 3/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1943 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 4ms/step - loss: 2.1801 - accuracy: 0.1064
Epoch 5/200
10/10 [==============================] - 0s 5ms/step - loss: 2.0655 - accuracy: 0.2340
Epoch 6/200
10/10 [==============================] - 0s 4ms/step - loss: 2.0754 - accuracy: 0.1489
Epoch 7/200
10/10 [==============================] - 0s 4ms/step - loss: 1.9488 - accuracy: 0.4255
Epoch 8/200
10/10 [==============================] - 0s 5ms/step - loss: 1.9632 - accuracy: 0.3617
Epoch 9/200
10/10 [==============================] - 0s 4ms/step - loss: 1.8482 - accuracy: 0.5532
Epoch 10/200
10/10 [==============================] - 0s 5ms/step - loss: 1.833

In [9]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')   # Load the pre-trained chatbot model
import json
import random
intents = json.loads(open('latestintent.json', encoding="utf8").read())  # Load intents from JSON file
# Load words and classes from pickle files
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [11]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']   # Extract the predicted intent tag
    list_of_intents = intents_json['intents']   # Extract all intents from the JSON object
    for i in list_of_intents:
        if(i['tag']== tag):    # Find the matching intent tag
            result = random.choice(i['responses'])      # Randomly select a response from the matched intent
            break 
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)   # Predict the intent of the input message using a classifier
    res = getResponse(ints, intents)    # Retrieve a response based on the predicted intent
    return res     # Return the generated response

In [ ]:
import tkinter
from tkinter import *

# Function to send the message
def send():
    # Get the message from the EntryBox, strip any extra whitespace, and clear the EntryBox
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

     # If the message is not empty
    if msg != '':
         # Enable ChatLog for editing
        ChatLog.config(state=NORMAL)
         # Insert user's message into ChatLog
        ChatLog.insert(END, "You: " + msg + '\n\n')
         # Configure text properties for user's message
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        # Get response from the chatbot based on user's message
        res = chatbot_response(msg)
        # Scroll ChatLog to the bottom to show the latest messages
        ChatLog.insert(END, "Bot: " + res + '\n\n')
            
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
 

# Create a new tkinter window
base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

# Create ChatLog widget to display messages
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 77ms/step
